In [1]:
# !pip install konlpy
import os
import json
from konlpy.tag import Komoran
import pandas as pd
tokenizer = Komoran()

Komoran 라이브러리 활용하여 문장별 형태소 tokenize. 후 주격조사, 목적격 조사 등 중요도가 떨어지는 형태소들을 제외한 핵심 형태소 추출. 문장별 핵심 형태소 길이 분포 파악.

In [2]:
path = './치료' 
file_list = os.listdir(path)
# 경로 내에 파일을 모두 불러옴

json_file_list = [file for file in file_list if file.endswith('.json')]
#json 확장자를 가진 파일로 새로운 리스트 생성

In [5]:
from pprint import pprint 
with open('치료/HC-Q-1124657.json') as f:
    jdata = json.load(f) 
    
pprint(jdata)

{'disease_category': '감염성질환',
 'disease_name': {'eng': 'Tuberculosis', 'kor': '결핵'},
 'entities': [{'entity': '질환명', 'id': 0, 'position': 1, 'text': '결핵'}],
 'fileName': 'HC-Q-1124657',
 'intention': '치료',
 'num_of_words': 7,
 'participantsInfo': {'age': '20대',
                      'gender': '여성',
                      'history': False,
                      'occupation': '사무직',
                      'participantID': 'QC298',
                      'rPlace': '서울/경기/인천'},
 'question': '폐결핵을 치료하는 데에는 어떤 치료 옵션이 효과적일까요?'}


In [ ]:
def loadLinesAndConversations(fileName):
    lines = {}
    conversations = {}
    with open(fileName, 'r', encoding='utf-8') as f:
        for line in f:
            lineJson = json.loads(line)
            # 필드를 추출하여 line 객체를 구성합니다
            lineObj = {}
            lineObj["intention"] = lineJson["intention"]
            lineObj["question"] = lineJson["question"]
            lineObj["text"] = lineJson["text"]
            lines[lineObj['lineID']] = lineObj

            # 필드를 추출하여 conversation 객체를 구성합니다
            if lineJson["conversation_id"] not in conversations:
                convObj = {}
                convObj["conversationID"] = lineJson["conversation_id"]
                convObj["movieID"] = lineJson["meta"]["movie_id"]
                convObj["lines"] = [lineObj]
            else:
                convObj = conversations[lineJson["conversation_id"]]
                convObj["lines"].insert(0, lineObj)
            conversations[convObj["conversationID"]] = convObj

    return lines, conversations

In [ ]:


# conversation들에서 문장 쌍을 추출합니다
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations.values():
        # 대화를 이루는 각 대사에 대해 반복문을 수행합니다
        # 대화의 마지막 대사는 (그에 대한 응답이 없으므로) 무시합니다
        for i in range(len(conversation["lines"]) - 1):
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # 잘못된 샘플은 제거합니다(리스트가 하나라도 비어 있는 경우)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs